In [ ]:
# Imports
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
from tqdm import tqdm
import pandas as pd

from geopy.geocoders import Nominatim

In [44]:
# スクレイピングするURLリストを作成
page_num = list(range(1, 8))
urls = [f"https://eitaikuyou.net/search/page/{p}" for p in page_num]

In [48]:
# 寺院リストの作成
temple_list = []
for url in tqdm(urls):
    response = requests.get(url)
    sleep(1)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards =soup.find_all('div', class_='temple-card')
    temples = []
    for card in cards:
        temple_name = card.find('h2').text.strip()
        full_address = card.find_all('li', class_='addr-info-list__item')[-1].text.strip()
        match = re.search(r'\d', full_address)
        if match:
            address = full_address[:match.start()]
        else:
            address = full_address
        item = {}
        item['寺院名'] = temple_name
        item['完全住所'] = full_address
        item['住所'] = address
        temples.append(item)
    temple_list.extend(temples)
df = pd.DataFrame(temple_list)

100%|██████████| 7/7 [00:09<00:00,  1.38s/it]


In [54]:
# 緯度経度の取得
geolocator = Nominatim(user_agent="geoapi")

latitudes = []
longitudes = []

for address in tqdm(df['住所'], desc="緯度経度取得中"):
    try:
        location = geolocator.geocode(address)
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
    except:
        latitudes.append(None)
        longitudes.append(None)
        sleep(1)  # レート制限回避

df['緯度'] = latitudes
df['経度'] = longitudes
# 住所リストをファイルに出力
df.to_csv('temple_addresses.csv', index=False)

緯度経度取得中: 100%|██████████| 93/93 [02:35<00:00,  1.68s/it]
